In [ ]:
## This notebook is used to average the monthly ERA5 temperature data to get yearly data
## Created by Kendra Herweck

In [3]:
import netCDF4 as nc
import numpy as np
import datetime as dt  # Python standard library datetime  module
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from astropy.table import Table
from astropy.io import ascii
from pandas import read_csv
from matplotlib.dates import DateFormatter
from pandas import DataFrame
from pandas import Grouper
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import xarray as xr
from glob import glob

## Average Monthly to get Yearly

In [5]:
filename1 = 'Data/Temperature-Data/ERA/adaptor.mars.internal-1624559809.1844091-5701-11-9322669f-b1dc-41e5-88ef-d6eeb6ddb4d9_1950-1978.nc'
ds1 = xr.open_dataset(filename1)
filename2 = 'Data/Temperature-Data/ERA/adaptor.mars.internal-1625059187.03663-23573-8-f14d887b-5fb1-4545-bf67-78ee0bf9b127_1979-2020.nc'
ds2 = xr.open_dataset(filename2)
ds = xr.merge([ds1,ds2])

print(ds)

<xarray.Dataset>
Dimensions:    (latitude: 361, longitude: 1440, time: 284)
Coordinates:
  * time       (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2020-12-01
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
Data variables:
    t          (time, latitude, longitude) float32 241.2 241.2 ... 298.9 298.8


In [8]:
ds_yearly = ds.groupby('time.year').mean('time',skipna=True)
print(ds_yearly)

<xarray.Dataset>
Dimensions:    (latitude: 361, longitude: 1440, year: 71)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
  * year       (year) int64 1950 1951 1952 1953 1954 ... 2017 2018 2019 2020
Data variables:
    t          (year, latitude, longitude) float32 243.9 243.9 ... 299.8 299.7


In [19]:
for i in range(0,len(ds_yearly['year'])): ##len(ds_yearly['year'])
    year = ds_yearly['year'][i].values
    filename = 'Data/Temperature-Data/ERA/Yearly/ERA-Temperature-{}.nc'.format(year)
    print(filename)
    f = nc.Dataset(filename, 'w')
    f.createDimension('time', 1)
    f.createDimension('y', len(ds_yearly['latitude'][0:321]))
    f.createDimension('x', len(ds_yearly['longitude']))
    lats = f.createVariable('lat', float, ('y', ), zlib=True)
    #print(lats.shape)
    lons = f.createVariable('lon', float, ('x', ), zlib=True)
    temp = f.createVariable('temp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)
    time = f.createVariable('time',float,('time'),zlib=True)
    
    lat_out  = np.array(ds_yearly.variables['latitude'][0:321])
    #print(lat_out.shape)
    lon_out  = ds_yearly.variables['longitude']
    #print(lat_out)
   # wind = ds_test['u'][i,0:321,:].values
    #print(lon_out.shape)
    lats[:] = lat_out
    lons[:] = lon_out
    temp[:] = ds_yearly['t'][i,0:321,:].values
    time[:] = ds_yearly['year'][i].values
    #wind_data = np.array(ds_test['u'][i,0:321,:].values)
    #print(wind_data.shape)
    #print(wind)
    #data = []
    #for i in range (0, len(lat_out)):
    #    for j in range(0,len(lon_out)):
    #        mean = np.nanmean(combined_ds['U'][0:,0,i,j].values)
    #        data.append(mean)
    #data_out = np.array(data)      ##1d array of data
    #data_out.shape = (len(lat_out),len(lon_out))     ##reshape to 2d array                 
   # f['U'][:,:] = wind_data
   # f['lat'][:] = lat_out
   # f['lon'][:] = lon_out
    f.close()

Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1950.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1951.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1952.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1953.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1954.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1955.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1956.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1957.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1958.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1959.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1960.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1961.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1962.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1963.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1964.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1965.nc
Data/Temperature-Data/ERA/Yearly/ERA-Temperature-1966.nc
Data/Temperature-Data/ERA/Yearl

## Climatological Avg

In [ ]:
filename_list=[]
for i in range(1980,2021):
    filename = 'Data/Temperature-Data/ERA/Yearly/ERA-Temperature-{}.nc'.format(i)
    filename_list.append(filename)

#print(filename_list)
combined_ds = xr.open_mfdataset(filename_list)
#print(combined_ds)

f = nc.Dataset('Climatology_ERA-Temperature_1000hPa', 'w')
f.createDimension('time', None)
f.createDimension('y', len(combined_ds['lat'][0,:]))
f.createDimension('x', len(combined_ds['lon'][0,:]))
#f.createDimension('level',len(combined_ds['lev']))
lats = f.createVariable('lat', float, ('y', ), zlib=True)
lons = f.createVariable('lon', float, ('x', ), zlib=True)
#levs = f.createVariable('lev',float, ('level', ), zlib= True)
temp = f.createVariable('temp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)

# create latitude and longitude 1D arrays
lat_out  = combined_ds.variables['lat'][0,:]
lon_out  = combined_ds.variables['lon'][0,:]

data = []
for i in range (0, len(lat_out)):
    print(lat_out[i].values)
    for j in range(0,len(lon_out)):
        mean = np.nanmean(combined_ds['temp'][:,i,j].values) #time,lat,lon
        data.append(mean)
data_out = np.array(data)      ##1d array of data
data_out.shape = (len(lat_out),len(lon_out))     ##reshape to 2d array                 
f['temp'][:] = data_out
f['lat'][:] = lat_out
f['lon'][:] = lon_out
# close file to write on disk
f.close()

## Low SIC Avg

In [ ]:
##years: 1950,1951,1952,1955,1958,1959,1961,1964,1966,1968,1970,1971,1974,1976,1977,1979,1981,1984,1987,1989,1990,1993,
##       1995,1997,1998,1999,2002,2005,2007,2010,2011,2012,2015,2016,2019,2020

filename_list=[]
for i in (1950,1951,1952,1955,1958,1959,1961,1964,1966,1968,1970,1971,1974,1976,1977,1979,1981,1984,1987,1989,1990,1993,
          1995,1997,1998,1999,2002,2005,2007,2010,2011,2012,2015,2016,2019,2020):
    filename = 'Data/Temperature-Data/ERA/Yearly/ERA-Temperature-{}.nc'.format(i)
    filename_list.append(filename)

#print(filename_list)
combined_ds = xr.open_mfdataset(filename_list)
#print(combined_ds)

f = nc.Dataset('LowSIC_ERA-Temperature_1000hPa', 'w')
f.createDimension('time', None)
f.createDimension('y', len(combined_ds['lat'][0,:]))
f.createDimension('x', len(combined_ds['lon'][0,:]))
#f.createDimension('level',len(combined_ds['lev']))
lats = f.createVariable('lat', float, ('y', ), zlib=True)
lons = f.createVariable('lon', float, ('x', ), zlib=True)
#levs = f.createVariable('lev',float, ('level', ), zlib= True)
temp = f.createVariable('temp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)

# create latitude and longitude 1D arrays
lat_out  = combined_ds.variables['lat'][0,:]
lon_out  = combined_ds.variables['lon'][0,:]

data = []
for i in range (0, len(lat_out)):
    print(lat_out[i].values)
    for j in range(0,len(lon_out)):
        mean = np.nanmean(combined_ds['temp'][:,i,j].values) #time,lat,lon
        data.append(mean)
data_out = np.array(data)      ##1d array of data
data_out.shape = (len(lat_out),len(lon_out))     ##reshape to 2d array                 
f['temp'][:] = data_out
f['lat'][:] = lat_out
f['lon'][:] = lon_out
# close file to write on disk
f.close()

## Low SIC & EP Nino Avg

In [37]:
##Years: 1952,1970,1977,1987,1998,2007,2012

filename_list=[]
for i in (1952,1970,1977,1987,1998,2007,2012):
    filename = 'Data/Temperature-Data/ERA/Yearly/ERA-Temperature-{}.nc'.format(i)
    filename_list.append(filename)

#print(filename_list)
combined_ds = xr.open_mfdataset(filename_list)
#print(combined_ds)

f = nc.Dataset('EPNino-LowSIC_ERA-Temperature_1000hPa', 'w')
f.createDimension('time', None)
f.createDimension('y', len(combined_ds['lat'][0,:]))
f.createDimension('x', len(combined_ds['lon'][0,:]))
#f.createDimension('level',len(combined_ds['lev']))
lats = f.createVariable('lat', float, ('y', ), zlib=True)
lons = f.createVariable('lon', float, ('x', ), zlib=True)
#levs = f.createVariable('lev',float, ('level', ), zlib= True)
temp = f.createVariable('temp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)

# create latitude and longitude 1D arrays
lat_out  = combined_ds.variables['lat'][0,:]
lon_out  = combined_ds.variables['lon'][0,:]

data = []
for i in range (0, len(lat_out)):
    print(lat_out[i].values)
    for j in range(0,len(lon_out)):
        mean = np.nanmean(combined_ds['temp'][:,i,j].values) #time,lat,lon
        data.append(mean)
data_out = np.array(data)      ##1d array of data
data_out.shape = (len(lat_out),len(lon_out))     ##reshape to 2d array                 
f['temp'][:] = data_out
f['lat'][:] = lat_out
f['lon'][:] = lon_out
# close file to write on disk
f.close()

90.0
89.75
89.5
89.25
89.0
88.75
88.5
88.25
88.0
87.75
87.5
87.25
87.0
86.75
86.5
86.25
86.0
85.75
85.5
85.25
85.0
84.75
84.5
84.25
84.0
83.75
83.5
83.25
83.0
82.75
82.5
82.25
82.0
81.75
81.5
81.25
81.0
80.75
80.5
80.25
80.0
79.75
79.5
79.25
79.0
78.75
78.5
78.25
78.0
77.75
77.5
77.25
77.0
76.75
76.5
76.25
76.0
75.75
75.5
75.25
75.0
74.75
74.5
74.25
74.0
73.75
73.5
73.25
73.0
72.75
72.5
72.25
72.0
71.75
71.5
71.25
71.0
70.75
70.5
70.25
70.0
69.75
69.5
69.25
69.0
68.75
68.5
68.25
68.0
67.75
67.5
67.25
67.0
66.75
66.5
66.25
66.0
65.75
65.5
65.25
65.0
64.75
64.5
64.25
64.0
63.75
63.5
63.25
63.0
62.75
62.5
62.25
62.0
61.75
61.5
61.25
61.0
60.75
60.5
60.25
60.0
59.75
59.5
59.25
59.0
58.75
58.5
58.25
58.0
57.75
57.5
57.25
57.0
56.75
56.5
56.25
56.0
55.75
55.5
55.25
55.0
54.75
54.5
54.25
54.0
53.75
53.5
53.25
53.0
52.75
52.5
52.25
52.0
51.75
51.5
51.25
51.0
50.75
50.5
50.25
50.0
49.75
49.5
49.25
49.0
48.75
48.5
48.25
48.0
47.75
47.5
47.25
47.0
46.75
46.5
46.25
46.0
45.75
45.5
45.25
45.0
44.75

In [36]:
f.close()

## Low SIC & CP Nino Avg

In [38]:
#Years: 1958,1959,1964,1966,1995,2005,2010,2015,2016,2020

filename_list=[]
for i in (1958,1959,1964,1966,1995,2005,2010,2015,2016,2020):
    filename = 'Data/Temperature-Data/ERA/Yearly/ERA-Temperature-{}.nc'.format(i)
    filename_list.append(filename)

#print(filename_list)
combined_ds = xr.open_mfdataset(filename_list)
#print(combined_ds)

f = nc.Dataset('CPNino-LowSIC_ERA-Temperature_1000hPa', 'w')
f.createDimension('time', None)
f.createDimension('y', len(combined_ds['lat'][0,:]))
f.createDimension('x', len(combined_ds['lon'][0,:]))
#f.createDimension('level',len(combined_ds['lev']))
lats = f.createVariable('lat', float, ('y', ), zlib=True)
lons = f.createVariable('lon', float, ('x', ), zlib=True)
#levs = f.createVariable('lev',float, ('level', ), zlib= True)
temp = f.createVariable('temp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)

# create latitude and longitude 1D arrays
lat_out  = combined_ds.variables['lat'][0,:]
lon_out  = combined_ds.variables['lon'][0,:]

data = []
for i in range (0, len(lat_out)):
    print(lat_out[i].values)
    for j in range(0,len(lon_out)):
        mean = np.nanmean(combined_ds['temp'][:,i,j].values) #time,lat,lon
        data.append(mean)
data_out = np.array(data)      ##1d array of data
data_out.shape = (len(lat_out),len(lon_out))     ##reshape to 2d array                 
f['temp'][:] = data_out
f['lat'][:] = lat_out
f['lon'][:] = lon_out
# close file to write on disk
f.close()

90.0
89.75
89.5
89.25
89.0
88.75
88.5
88.25
88.0
87.75
87.5
87.25
87.0
86.75
86.5
86.25
86.0
85.75
85.5
85.25
85.0
84.75
84.5
84.25
84.0
83.75
83.5
83.25
83.0
82.75
82.5
82.25
82.0
81.75
81.5
81.25
81.0
80.75
80.5
80.25
80.0
79.75
79.5
79.25
79.0
78.75
78.5
78.25
78.0
77.75
77.5
77.25
77.0
76.75
76.5
76.25
76.0
75.75
75.5
75.25
75.0
74.75
74.5
74.25
74.0
73.75
73.5
73.25
73.0
72.75
72.5
72.25
72.0
71.75
71.5
71.25
71.0
70.75
70.5
70.25
70.0
69.75
69.5
69.25
69.0
68.75
68.5
68.25
68.0
67.75
67.5
67.25
67.0
66.75
66.5
66.25
66.0
65.75
65.5
65.25
65.0
64.75
64.5
64.25
64.0
63.75
63.5
63.25
63.0
62.75
62.5
62.25
62.0
61.75
61.5
61.25
61.0
60.75
60.5
60.25
60.0
59.75
59.5
59.25
59.0
58.75
58.5
58.25
58.0
57.75
57.5
57.25
57.0
56.75
56.5
56.25
56.0
55.75
55.5
55.25
55.0
54.75
54.5
54.25
54.0
53.75
53.5
53.25
53.0
52.75
52.5
52.25
52.0
51.75
51.5
51.25
51.0
50.75
50.5
50.25
50.0
49.75
49.5
49.25
49.0
48.75
48.5
48.25
48.0
47.75
47.5
47.25
47.0
46.75
46.5
46.25
46.0
45.75
45.5
45.25
45.0
44.75

## All CP Nino Years

In [ ]:
##Years: 1954,1958,1959,1964,1966,1969,1978,1988,1992,1995,2003,2005,2010,2015,2016,2020

filename_list=[]
for i in (1954,1958,1959,1964,1966,1969,1978,1988,1992,1995,2003,2005,2010,2015,2016,2020):
    filename = 'Data/Temperature-Data/ERA/Yearly/ERA-Temperature-{}.nc'.format(i)
    filename_list.append(filename)

#print(filename_list)
combined_ds = xr.open_mfdataset(filename_list)
#print(combined_ds)

f = nc.Dataset('CPNino_ERA-Temperature_1000hPa', 'w')
f.createDimension('time', None)
f.createDimension('y', len(combined_ds['lat'][0,:]))
f.createDimension('x', len(combined_ds['lon'][0,:]))
#f.createDimension('level',len(combined_ds['lev']))
lats = f.createVariable('lat', float, ('y', ), zlib=True)
lons = f.createVariable('lon', float, ('x', ), zlib=True)
#levs = f.createVariable('lev',float, ('level', ), zlib= True)
temp = f.createVariable('temp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)

# create latitude and longitude 1D arrays
lat_out  = combined_ds.variables['lat'][0,:]
lon_out  = combined_ds.variables['lon'][0,:]

data = []
for i in range (0, len(lat_out)):
    print(lat_out[i].values)
    for j in range(0,len(lon_out)):
        mean = np.nanmean(combined_ds['temp'][:,i,j].values) #time,lat,lon
        data.append(mean)
data_out = np.array(data)      ##1d array of data
data_out.shape = (len(lat_out),len(lon_out))     ##reshape to 2d array                 
f['temp'][:] = data_out
f['lat'][:] = lat_out
f['lon'][:] = lon_out
# close file to write on disk
f.close()

## All EP Nino Years

In [ ]:
##Years: 1952,1970,1973,1977,1983,1987,1998,2007,2012

filename_list=[]
for i in (1952,1970,1973,1977,1983,1987,1998,2007,2012):
    filename = 'Data/Temperature-Data/ERA/Yearly/ERA-Temperature-{}.nc'.format(i)
    filename_list.append(filename)

#print(filename_list)
combined_ds = xr.open_mfdataset(filename_list)
#print(combined_ds)

f = nc.Dataset('EPNino_ERA-Temperature_1000hPa', 'w')
f.createDimension('time', None)
f.createDimension('y', len(combined_ds['lat'][0,:]))
f.createDimension('x', len(combined_ds['lon'][0,:]))
#f.createDimension('level',len(combined_ds['lev']))
lats = f.createVariable('lat', float, ('y', ), zlib=True)
lons = f.createVariable('lon', float, ('x', ), zlib=True)
#levs = f.createVariable('lev',float, ('level', ), zlib= True)
temp = f.createVariable('temp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)

# create latitude and longitude 1D arrays
lat_out  = combined_ds.variables['lat'][0,:]
lon_out  = combined_ds.variables['lon'][0,:]

data = []
for i in range (0, len(lat_out)):
    print(lat_out[i].values)
    for j in range(0,len(lon_out)):
        mean = np.nanmean(combined_ds['temp'][:,i,j].values) #time,lat,lon
        data.append(mean)
data_out = np.array(data)      ##1d array of data
data_out.shape = (len(lat_out),len(lon_out))     ##reshape to 2d array                 
f['temp'][:] = data_out
f['lat'][:] = lat_out
f['lon'][:] = lon_out
# close file to write on disk
f.close()

90.0
89.75
89.5
89.25
89.0
88.75
88.5
88.25
88.0
87.75
87.5
87.25
87.0
86.75
86.5
86.25
86.0
85.75
85.5
85.25
85.0
84.75
84.5
84.25
84.0
83.75
83.5
83.25
83.0
82.75
82.5
82.25
82.0
81.75
81.5
81.25
81.0
80.75
80.5
80.25
80.0
79.75
79.5
79.25
79.0
78.75
78.5
78.25
78.0
77.75
77.5
77.25
77.0
76.75
76.5
76.25
76.0
75.75
75.5
75.25
75.0
74.75
74.5
74.25
74.0
73.75
73.5
73.25
73.0
72.75
72.5
72.25
72.0
71.75
71.5
71.25
71.0
70.75
70.5
70.25
70.0
69.75
69.5
69.25
69.0
68.75
68.5
68.25
68.0
67.75
67.5
67.25
67.0
66.75
66.5
66.25
66.0
65.75
65.5
65.25
65.0
64.75
64.5
64.25
64.0
63.75
63.5


# 2 M Temp

## Average Monthly to get Yearly

In [4]:
filename1 = 'Data/Temperature-Data/ERA/2M-Temp/adaptor.mars.internal-1625493241.2038293-23274-15-c4ecc21e-ed0d-4b24-8383-d490f057aa47_1950-1978.nc'
ds1 = xr.open_dataset(filename1)
filename2 = 'Data/Temperature-Data/ERA/2M-Temp/adaptor.mars.internal-1625493245.9144063-10037-9-5afb427a-c919-417f-a184-a3d74f42a9e9_1979-2020.nc'
ds2 = xr.open_dataset(filename2)
ds = xr.merge([ds1,ds2])

print(ds)

<xarray.Dataset>
Dimensions:    (latitude: 321, longitude: 1440, time: 286)
Coordinates:
  * time       (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2021-02-01
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... 10.5 10.25 10.0
Data variables:
    t2m        (time, latitude, longitude) float32 240.8 240.8 ... 299.5 299.5


In [5]:
ds_yearly = ds.groupby('time.year').mean('time',skipna=True)
print(ds_yearly)

<xarray.Dataset>
Dimensions:    (latitude: 321, longitude: 1440, year: 72)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... 10.5 10.25 10.0
  * year       (year) int64 1950 1951 1952 1953 1954 ... 2018 2019 2020 2021
Data variables:
    t2m        (year, latitude, longitude) float32 242.8 242.8 ... 299.6 299.6


In [7]:
for i in range(0,len(ds_yearly['year'])): ##len(ds_yearly['year'])
    year = ds_yearly['year'][i].values
    filename = 'Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-{}.nc'.format(year)
    print(filename)
    f = nc.Dataset(filename, 'w')
    f.createDimension('time', 1)
    f.createDimension('y', len(ds_yearly['latitude']))
    f.createDimension('x', len(ds_yearly['longitude']))
    lats = f.createVariable('lat', float, ('y', ), zlib=True)
    #print(lats.shape)
    lons = f.createVariable('lon', float, ('x', ), zlib=True)
    temp = f.createVariable('temp', float, ('y', 'x'), zlib=True, least_significant_digit=1, fill_value=0)
    time = f.createVariable('time',float,('time'),zlib=True)
    
    lat_out  = np.array(ds_yearly.variables['latitude'])
    #print(lat_out.shape)
    lon_out  = ds_yearly.variables['longitude']
    #print(lat_out)
   # wind = ds_test['u'][i,0:321,:].values
    #print(lon_out.shape)
    lats[:] = lat_out
    lons[:] = lon_out
    temp[:] = ds_yearly['t2m'][i,:,:].values
    time[:] = ds_yearly['year'][i].values
    #wind_data = np.array(ds_test['u'][i,0:321,:].values)
    #print(wind_data.shape)
    #print(wind)
    #data = []
    #for i in range (0, len(lat_out)):
    #    for j in range(0,len(lon_out)):
    #        mean = np.nanmean(combined_ds['U'][0:,0,i,j].values)
    #        data.append(mean)
    #data_out = np.array(data)      ##1d array of data
    #data_out.shape = (len(lat_out),len(lon_out))     ##reshape to 2d array                 
   # f['U'][:,:] = wind_data
   # f['lat'][:] = lat_out
   # f['lon'][:] = lon_out
    f.close()

Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1950.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1951.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1952.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1953.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1954.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1955.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1956.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1957.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1958.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1959.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1960.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1961.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1962.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1963.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-1964.nc
Data/Temperature-Data/ERA/Yearly-2M/ERA-2M-Temperature-